# Train a language model (Masked Language Modelling) from scratch using Huggingface Transformers and a custom tokenizer

### Inspired from the great notebook by Huggingfce (link to blogpost [link](https://huggingface.co/blog/how-to-train)).

# Brief Introduction
This blog post is the first part of a series where we want to create a product names generator using a transformer model. For a few weeks I was investigating different models and alternatives in Huggingface to train a text generation model. We have a short list of products with their description and our goal is to obtain the name of the product. I did some experiments with the Transformer model in Tensorflow as well as the T5 summarizer. Finally, in order to deepen the use of Huggingface transformers, I decided to approach the problem with a somewhat more complex approach, an encoder decoder model. Maybe it was not the best option but I wanted to learn new things about huggingface Transformers. In the next post of the series we will introduce you deeper in this concept.

Here, in this first part, we will show how to train a tokenizer from scratch and how to use Masked Language Modeling technique to create a RoBERTa model. This personalized model will become the base model for our future encoder Decoder model.

# Our Solution
For our experiment we are going to train from scratch a RoBERTa model, it will become the encoder and the decoder of a future model. But our domain is very specific, words and concepts about clothes, shapes, colors, … Therefore, we are interested in defining our own tokenizer created from our specific vocabulary, avoiding to include more common words from other domains or use cases which are irrelevant for our final purpose.

*We can describe our training phase in three main steps*:
- Create and train a byte-level, **Byte-pair encoding tokenizer** with the same special tokens as RoBERTa
- Train a RoBERTa model from scratch using **Masked Language Modeling**, MLM.
- Warm start and **fine tune an encoder decoder model** based on our RoBERTa pretrained model.

In this post we’ll demo how to train a “small” RoBERTa model (6 layers, 768 hidden size, 12 attention heads) – that’s the same number of layers & heads as DistilBERT – on a language from a clothing shop. And in a next notebook, we’ll then fine-tune the model on a downstream task of text generation.


# Loading the libraries

In [52]:
import os
import pandas as pd
import tqdm
import math

# Loading the datasets

As we mentioned before, our dataset contains around 31.000 items, about clothes from an important retailer, including a long product description and a short product name, our target variable. First, we execute a exploratory data analysis and we can observe that the count of rows with outliers values is a small number. The count of words looks like a left skewed distribution, 75% of rows in the range 50–60 words and a maximum about 125 words. The target variable contains about 3 to 6 words.

In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Set the variables to the data folders:

In [3]:
#Set the path to the data folder, datafile and output folder and files
root_folder = '/content/drive/My Drive/'
data_folder = os.path.abspath(os.path.join(root_folder, 'datasets/text_gen_product_names'))
model_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names/RoBERTaMLM'))
output_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names'))
tokenizer_folder = os.path.abspath(os.path.join(root_folder, 'Projects/text_generation_names/TokRoBERTa'))

test_filename='cl_test_descriptions.csv'
datafile= 'product_names_desc_cl_train.csv'
outputfile = 'submission.csv'

datafile_path = os.path.abspath(os.path.join(data_folder,datafile))
testfile_path = os.path.abspath(os.path.join(data_folder,test_filename))
outputfile_path = os.path.abspath(os.path.join(output_folder,outputfile))

Load the train datafile with the product descriptions and names:

In [4]:
# Load the train dataset
train_df=pd.read_csv(datafile_path, header=0, usecols=[0,1])
# Show the count of rows
print('Num Examples: ',len(train_df))
print('Null Values\n', train_df.isna().sum())
# Drop rows with Null values 
train_df.dropna(inplace=True)
print('Num Examples: ',len(train_df))

Num Examples:  31593
Null Values
 name           44
description     1
dtype: int64
Num Examples:  31548


Then, we read the test dataset:

In [5]:
# Load the test dataset 
test_df=pd.read_csv(testfile_path, header=0)
print('Num Examples: ',len(test_df))
print('Null Values\n', test_df.isna().sum())
# there are no null values

Num Examples:  1441
Null Values
 description    0
dtype: int64


# Build a Tokenizer


## Create the dataset to train a tokenizer

*To train a tokenizer we need to save our dataset in a bunch of text files*. We create a plain text file for every description value and we will split each sample using a newline character. We include both the train and test dataset:

In [6]:
# Drop the files from the output dir
txt_files_dir = "./text_split"
!rm -rf {txt_files_dir}
!mkdir {txt_files_dir}

In [7]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e) 
      i+=1
    # Return the last ID
    return i


Include the training dataset to the main text file:

In [8]:
data = train_df["description"]
# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)

31548


Also include the test dataset to the text file:

In [9]:
data = test_df["description"]
# Removing the end of line character \n
data = data.replace("\n"," ")
print(len(data))
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

1441
32989


**Include the target variable for training** NOOOO¿?

In [ ]:
data = train_df["name"]
data = data.replace("\n"," ")
print(len(data))
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

31548
64537


## Train the tokenizer

The Stanford NLP group define the tokenization as:

"*Given a character sequence and a defined document unit, tokenization is the task of chopping it up into pieces, called tokens , perhaps at the same time throwing away certain characters, such as punctuation.*"

A tokenizer breaks a string of characters, usually sentences of text, into tokens, an integer representation of the token, usually by looking for whitespace (tabs, spaces, new lines). It usually splits a sentence into words but there are many options like subwords.

We will use a **byte-level Byte-pair encoding tokenizer**, byte pair encoding (BPE) is a simple form of data compression in which the most common pair of consecutive bytes of data is replaced with a byte that does not occur within that data. The benefit of this method is that it will start building its vocabulary from an alphabet of single chars, so all words will be decomposable into tokens. We can avoid the presence of unknown (UNK) tokens.

A great explanation on tokenizers can be found on the huggingface documentation, https://huggingface.co/transformers/tokenizer_summary.html.

In [10]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
!pip install datasets==1.0.2

Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9j2gofz2
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-9j2gofz2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 30.8 MB/s 
     |████████████████████████████████| 636 kB 63.6 MB/s 
     |████████████████████████████████| 3.3 MB 67.0 MB/s 
  Created wheel for transformers: filename=transformers-4.10.0.dev0-py3-none-any.whl size=2587137 sha256=0ddb51a33439fae45846cd69ce6cf89f7d2f41770c7990cdb2e13347c94eee8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-iargkcud/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation:

We choose to train a byte-level Byte-pair encoding tokenizer (the same as GPT-2), with the same special tokens as RoBERTa. Let’s pick its size to be 8,192 because our specific vocabulary is very limited and simple.

In [46]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

import torch
from torch.utils.data.dataset import Dataset

Now we can train our tokenizer on the text files containing our vocabulary, we need to specify the vocabulary size, the min frequency for a token to be included and the special tokens. We choose a vocab size of 8,192 and a min frequency of 2 (you can tune this value depending on your max vocabulary size). 

The special tokens depends on the model, for RoBERTa we include a short list: 
- \<s> or BOS, beginning Of Sentence
- \</s> or EOS, End Of Sentence
- \<pad> the padding token
- \<unk> the unknown token
- \<mask> the masking token.

In [12]:
%%time 
paths = [str(x) for x in Path(".").glob("text_split/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])

CPU times: user 2.12 s, sys: 842 ms, total: 2.97 s
Wall time: 1.69 s


In [13]:
tokenizer

Tokenizer(vocabulary_size=8192, model=ByteLevelBPE, add_prefix_space=False, lowercase=True, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

The count of samples is small and the tokenizer trains very fast. Now we can save the tokenizer to disk, later we will use it to train the language model:

In [14]:
#Save the Tokenizer to disk
tokenizer.save_model(tokenizer_folder)

['/content/drive/My Drive/Projects/text_generation_names/TokRoBERTa/vocab.json',
 '/content/drive/My Drive/Projects/text_generation_names/TokRoBERTa/merges.txt']

We now have both a `vocab.json`, which is a list of the most frequent tokens ranked by frequency and it is used to convert tokens to IDs, and a `merges.txt` file that maps texts to tokens.

```json
{
	"<s>": 0,
	"<pad>": 1,
	"</s>": 2,
	"<unk>": 3,
	"<mask>": 4,
	"!": 5,
	"\"": 6,
	"#": 7,
	"$": 8,
	"%": 9,
	"&": 10,
	"'": 11,
	"(": 12,
	")": 13,
	# ...
}

# merges.txt
l a
Ġ k
o n
Ġ la
t a
Ġ e
Ġ d
Ġ p
# ...
```

What is great is that our tokenizer is optimized for our very specific vocabulary. Compared to a generic tokenizer trained for English, more native words are represented by a single, unsplit token. 

Here’s  how you can use it in `tokenizers`, including handling the RoBERTa special tokens – of course, you’ll also be able to use it directly from `transformers`. We can instantiate our tokenizer using both files and test it with some text from our dataset.


In [15]:
# Create the tokenizer using vocab.json and mrege.txt files
tokenizer = ByteLevelBPETokenizer(
    os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
)

In [16]:
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

Let's show some examples:

In [17]:
tokenizer.encode("knit midi dress with vneckline straps.")

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [18]:
# Show the tokens created
tokenizer.encode("knit midi dress with vneckline straps.").tokens

['<s>',
 'knit',
 'Ġmidi',
 'Ġdress',
 'Ġwith',
 'Ġvneckline',
 'Ġstraps',
 '.',
 '</s>']

# Train a language model from scratch

**Update:** This section follows along the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, using our new [`Trainer`](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py) directly. Feel free to pick the approach you like best.

> We’ll train a RoBERTa-like model, which is a BERT-like with a couple of changes (check the [documentation](https://huggingface.co/transformers/model_doc/roberta.html) for more details). In summary: *It builds on BERT and modifies key hyperparameters, removing the next-sentence pretraining objective and training with much larger mini-batches and learning rates* .

As the model is BERT-like, we’ll train it on a task of **Masked language modeling**. It involves masking part of the input, about 10-20% of thre tokens, then learning a model to predict the missing tokens. MLM is often used within pretraining tasks, **to give models the opportunity to learn textual patterns from unlabeled data**. It can be fine tuned to a particular downstream task. The main benefit is that we do not need labeled data (hard to obtain), no text needs to be labeled by human labelers in order to predict the missing values.


We define some global parameters:

In [37]:
TRAIN_BATCH_SIZE = 16    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 8    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 15        # number of epochs to train (default: 10)
LEARNING_RATE = 1e-4    # learning rate (default: 0.001)
WEIGHT_DECAY = 0.01
SEED = 42               # random seed (default: 42)
MAX_LEN = 128
SUMMARY_LEN = 7

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Jul 22 10:57:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

##Define the model

We are going to train the model from scratch, not from a pretrained one. We create a model configuration for our RoBERTa model setting the main parameters:
- Vocabulary size
- Attention heads
- Hidden layers
- etc,

In [20]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

Finally let's initialize our model using the configuration file. As we are training from scratch, we only initialize from a config that define the architecture of the model but *not restoring previously trained weights*. The weights will be randomly initialized. 

In [21]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


Now let's recreate our tokenizer, using the tokenizer trained and saved in the previous step. We will use a `RoBERTaTokenizerFast` object and the `from_pretrained` method, to initialize our tokenizer.

In [38]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

In [34]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/drive/My Drive/Projects/text_generation_names/TokRoBERTa', vocab_size=8192, model_max_len=256, is_fast=True, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

## Building the training Dataset

We'll build a Pytorch dataset, subclassing the Dataset Class. The CustomDataset receives a Pandas Series with the `description` variable values and the tokenizer to encode those values. The Dataset returns a list of tokens for every product description in the Series.

In order to evaluate the model during training, we will generate a train dataset for training and a evaluation dataset.


https://ryanong.co.uk/2020/06/11/day-163-how-to-build-a-language-model-from-scratch-implementation/

In [47]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []
        
        for example in df.values:
            x=tokenizer.encode_plus(example, max_length = MAX_LEN, truncation=True, padding=True)
            self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

Concat the training and test dataset, only with the description column.

In [ ]:
# Concatenate the train dataset and the test dataset for language modelling
#df=pd.concat([train_df['description'], test_df['description']], axis=0)
#print('Total: ',len(df), len(train_df), len(test_df))


32989 31548 1441
0       towel with border with lines metallic thread .
1    printed bermuda shorts made technical fabric ....
2    bodysuit with shapewear effect . this bodysuit...
3    vneck with thin adjustable straps.height model...
4    puritan collar dress featuring long sleeves wi...
Name: description, dtype: object


Create the custom datasets, for training and evaluation:

In [48]:
# Create the train and evaluation dataset
train_dataset = CustomDataset(train_df['description'], tokenizer)
eval_dataset = CustomDataset(test_df['description'], tokenizer)

## Define the Data Collactor for masking our language

Like in the [`run_language_modeling.py`](https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py) script, we need to define a data_collator.

Once we have the dataset, a **Data Collator will helps us to mask our training texts**. This is just a small helper that will help us batch different samples of the dataset together into an object that PyTorch knows how to perform backprop on. Data collators are objects that will form a batch by using a list of dataset elements as input and may apply some processing like padding or random masking. The `DataCollatorForLanguageModeling` method allow us to set the probability with which to randomly mask tokens in the input.

In [41]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

## Initialize and train our Trainer

When we want to train a transformer model, the basic approach is to create a Trainer class that provides an API for feature-complete training and contains the basic training loop. First, we define the training arguments, there are many of them but the more relevant are
- `output_dir`, where the model artifacts will be saved
- `num_train_epochs`
- `per_device_train_batch_size`, the batch size


 and then the `Trainer` object is created with the arguments, the input dataset and the data collator defined:



In [49]:
from transformers import Trainer, TrainingArguments

print(model_folder)
# Define the training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #prediction_loss_only=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


/content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM


And now, we are ready to train our model 

In [50]:
# Train the model
trainer.train()

***** Running training *****
  Num examples = 31548
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 29580


Epoch,Training Loss,Validation Loss
1,4.333300,4.805030
2,3.096400,3.990580
3,2.608900,3.612991
4,2.407900,3.419896
5,2.145000,3.167123
6,2.005000,2.985954
7,1.894100,2.871262
8,1.807300,2.751488
9,1.713600,2.640090
10,1.611300,2.510171


***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
Saving model checkpoint to /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/checkpoint-8192
Configuration saved in /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/checkpoint-8192/config.json
Model weights saved in /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/checkpoint-8192/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8
Saving model checkpoint to /content/drive/My Drive/Projects/text_generation_names/RoBERTaML

TrainOutput(global_step=29580, training_loss=2.1221812471979127, metrics={'train_runtime': 3103.8529, 'train_samples_per_second': 152.462, 'train_steps_per_second': 9.53, 'total_flos': 6547575213342720.0, 'train_loss': 2.1221812471979127, 'epoch': 15.0})

As a result, we can watch how the loss is decreasing while training. We can evaluate our model on the validation set. The perplexity is high because we only have to make predictions for the masked tokens (which represent 15% of the total here).

In [53]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1441
  Batch size = 8


Perplexity: 11.75


## Save our final model and tokenizer to disk

Save the model and tokenizer ina way that they can be restored for a future downstream task, our encoder decoder model

In [54]:
trainer.save_model(model_folder)

Saving model checkpoint to /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM
Configuration saved in /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/config.json
Model weights saved in /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/pytorch_model.bin


# Checking the trained model using a Pipeline

Looking at the training and eval losses going down is not enough, we would like to apply our model to check if our language model is learning anything interesting. An easy way is via the FillMaskPipeline.

Pipelines are simple wrappers around tokenizers and models. **We can use the 'fill-mask' pipeline** where we input a sequence containing a masked token (<mask>) and it returns a list of the most probable filled sequences, with their probabilities.


In [55]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model_folder,
    tokenizer=tokenizer_folder
)

loading configuration file /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.10.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 8192
}

loading configuration file /content/drive/My Drive/Projects/text_generation_names/RoBERTaMLM/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1

In [56]:
# knit midi dress with vneckline
# =>
fill_mask("midi <mask> with vneckline.")

[{'score': 0.9493752717971802,
  'sequence': 'midi dress with vneckline.',
  'token': 482,
  'token_str': ' dress'},
 {'score': 0.02125515230000019,
  'sequence': 'midi skirt with vneckline.',
  'token': 769,
  'token_str': ' skirt'},
 {'score': 0.0033718657214194536,
  'sequence': 'midi sleeve with vneckline.',
  'token': 469,
  'token_str': ' sleeve'},
 {'score': 0.0026314768474549055,
  'sequence': 'midi vest with vneckline.',
  'token': 2315,
  'token_str': ' vest'},
 {'score': 0.00247795507311821,
  'sequence': 'midi crop with vneckline.',
  'token': 1693,
  'token_str': ' crop'}]

Ok, simple syntax/grammar works. Let’s try a slightly more interesting prompt:



In [57]:
# The test text: Round neck sweater with long sleeves
fill_mask("Round neck sweater with <mask> sleeves.")

[{'score': 0.7701749801635742,
  'sequence': 'Round neck sweater with long sleeves.',
  'token': 345,
  'token_str': ' long'},
 {'score': 0.20450818538665771,
  'sequence': 'Round neck sweater with short sleeves.',
  'token': 429,
  'token_str': ' short'},
 {'score': 0.004017259925603867,
  'sequence': 'Round neck sweater with turnup sleeves.',
  'token': 803,
  'token_str': ' turnup'},
 {'score': 0.0022616705391556025,
  'sequence': 'Round neck sweater with ruffled sleeves.',
  'token': 695,
  'token_str': ' ruffled'},
 {'score': 0.0019150367006659508,
  'sequence': 'Round neck sweater with cuffed sleeves.',
  'token': 722,
  'token_str': ' cuffed'}]